# TODO
- - -

1. 전처리
    - 숫자 제거
    - 소문자화
    - 불용어 제거
    - 구두점 제거
2. scikit-learn으로 vectorize(representation)
3. naive-bayes로 학습
    - 가우시안 분포, 베르누이 분포, 다항 분포 중 어떤 분포가 좋은지 파악

# Practice
- - -

In [2]:
!ls

181210_logistic_regression_01.ipynb advertising.csv
181210_logistic_regression_02.ipynb cache
181211_naivebayes_01.ipynb          sms_spam.csv
Social_Network_Ads.csv


In [3]:
import pandas as pd

In [6]:
data_raw = pd.read_csv('sms_spam.csv', encoding='ISO-8859-1')
data_raw.tail()

,type,text
5554,ham,You are a great role model. You are giving so ...
5555,ham,"Awesome, I remember the last time we got someb..."
5556,spam,"If you don't, your prize will go to another cu..."
5557,spam,"SMS. ac JSco: Energy is high, but u may not kn..."
5558,ham,Shall call now dear having food


### preprocessing

In [12]:
# lower()

data_raw.text = data_raw.text.apply(lambda x : x.lower())

In [24]:
corpus = data_raw.text.values
corpus = list(corpus)
corpus[:5]

['hope you are having a good week. just checking in',
 'k..give back my thanks.',
 'am also doing in cbe only. but have to pay.',
 'complimentary 4 star ibiza holiday or â£10,000 cash needs your urgent collection. 09066364349 now from landline not to lose out! box434sk38wp150ppm18+',
 'okmail: dear dave this is your final notice to collect your 4* tenerife holiday or #5000 cash award! call 09061743806 from landline. tcs sae box326 cw25wx 150ppm']

In [19]:
type(corpus), len(corpus)

(list, 5559)

In [ ]:
data_raw.type = data_raw.type.apply(lambda x : 1 if x == 'spam' else 0)

### vectorization

In [69]:
# countvectorizer

In [101]:
corpus[:10]

['hope you are having a good week. just checking in',
 'k..give back my thanks.',
 'am also doing in cbe only. but have to pay.',
 'complimentary 4 star ibiza holiday or â£10,000 cash needs your urgent collection. 09066364349 now from landline not to lose out! box434sk38wp150ppm18+',
 'okmail: dear dave this is your final notice to collect your 4* tenerife holiday or #5000 cash award! call 09061743806 from landline. tcs sae box326 cw25wx 150ppm',
 'aiya we discuss later lar... pick u up at 4 is it?',
 'are you this much buzy',
 'please ask mummy to call father',
 'marvel mobile play the official ultimate spider-man game (â£4.50) on ur mobile right now. text spider to 83338 for the game & we ll send u a free 8ball wallpaper',
 "fyi i'm at usf now, swing by the room whenever"]

In [25]:
from sklearn.feature_extraction.text import CountVectorizer

CV = CountVectorizer()
X_sparse = CV.fit_transform(corpus)
X_sparse

<5559x8674 sparse matrix of type '<class 'numpy.int64'>'
	with 73311 stored elements in Compressed Sparse Row format>

In [80]:
X_sparse.shape, type(X_sparse)

((5559, 8674), scipy.sparse.csr.csr_matrix)

In [79]:
X_dense = X_sparse.todense()
X_dense.shape, type(X_dense)

((5559, 8674), numpy.matrixlib.defmatrix.matrix)

In [81]:
X_array = X_sparse.toarray()
X_array.shape, type(X_array)

((5559, 8674), numpy.ndarray)

### Modeling: BernoulliNB

In [109]:
y = data_raw.type.values
X.shape, y.shape

((5559, 8674), (5559,))

In [82]:
from sklearn.naive_bayes import BernoulliNB
from sklearn.model_selection import cross_val_score

Bern = BernoulliNB()

### sparse matrix

In [96]:
%%time

# sparse
model_sparse = Bern.fit(X_sparse, y)
y_pred = model_sparse.predict(X_sparse)
result = np.mean(y == y_pred)
print("X_sparse result : %f \n" % (result))

X_sparse result : 0.987588 

CPU times: user 9.99 ms, sys: 2.55 ms, total: 12.5 ms
Wall time: 10.2 ms


In [97]:
%%time

# dense
model_dense = Bern.fit(X_dense, y)
y_pred = model_dense.predict(X_dense)
result = np.mean(y == y_pred)
print("X_dense result : %f \n" % (result))

X_dense result : 0.987588 

CPU times: user 1.85 s, sys: 1.44 s, total: 3.29 s
Wall time: 3.1 s


In [98]:
%%time

# array
model_array = Bern.fit(X_array, y)
y_pred = model_array.predict(X_array)
result = np.mean(y == y_pred)
print("X_array result : %f \n" % (result))

X_array result : 0.987588 

CPU times: user 1.89 s, sys: 1.57 s, total: 3.45 s
Wall time: 3.5 s


In [108]:
# cross validation

validation_score = []

for i in range(3, 13, 2):
    scores = cross_val_score(Bern, X_sparse, y, cv=i, scoring='accuracy')
    result = np.mean(scores)
    validation_score.append(result)
    print("cv : %d, X_array result : %f \n" % (i, result))

cv : 3, X_array result : 0.978054 

cv : 5, X_array result : 0.979672 

cv : 7, X_array result : 0.981470 

cv : 9, X_array result : 0.981290 

cv : 11, X_array result : 0.981828 



### Modeling: MultinomialNB

In [114]:
# load estimator
from sklearn.naive_bayes import MultinomialNB
Mult = MultinomialNB()

# cross validation
validation_score = []

for i in range(3, 9, 2):
    scores = cross_val_score(Mult, X_dense, y, cv=i, scoring='accuracy')
    result = np.mean(scores)
    validation_score.append(result)
    print("cv : %d, X_array result : %f \n" % (i, result))

cv : 3, X_array result : 0.979133 

cv : 5, X_array result : 0.980392 

cv : 7, X_array result : 0.980752 



### Modeling: GaussianNB

In [115]:
# load estimator
from sklearn.naive_bayes import GaussianNB
Gaus = GaussianNB()

# cross validation
validation_score = []

for i in range(3, 9, 2):
    scores = cross_val_score(Gaus, X_dense, y, cv=i, scoring='accuracy')
    result = np.mean(scores)
    validation_score.append(result)
    print("cv : %d, X_array result : %f \n" % (i, result))

cv : 3, X_array result : 0.902321 

cv : 5, X_array result : 0.896027 

cv : 7, X_array result : 0.895491 

